In [ ]:
import requests
import xmltodict

In [ ]:

url = "https://tds.hycom.org/thredds/catalogs/GOMu0.04/expt_90.1m000.xml"
with requests.get(url) as r:
        doc = xmltodict.parse(r.text)

In [ ]:
for ds in doc['catalog']['dataset']['dataset']['dataset']:
    if 'ALL' in ds['@name'] or 'FMRC' in ds['@name']: # restrict to Hindcast
        continue
    #print(ds['@name'])
    #print(ds['@ID'])
    #print(ds['@urlPath'])
    har = c3.HindcastArchive(
        **{
            "id": ds['@ID'],
            "dataset": {"id": "GOMu0.04_901m000_FMRC_1.0.1"},
            "name":ds['@ID'],
            "description": ds['@name'],
            "urlPath": ds['@urlPath']
            
          }
    ).upsert()
    print(har)
    
    
    
    
    

In [1]:
# Convenience function to download Hindcast file to local path
def downloadHindcast(da, opts, localPath='/tmp'):
    # Generate the thredds query url
    url = c3.HycomUtil.createThreddsUrl(da.urlPath, opts)
    # Download the file to c3 storage
    if opts.timeRange.start == opts.timeRange.end:
        filename = da.id + '-' + opts.timeRange.start.strftime("%Y-%m-%dT%H:%M:%SZ") + '.nc'
    else:
        filename = da.id + '-' + opts.timeRange.start.strftime("%Y-%m-%dT%H:%M:%SZ") + '-' + opts.timeRange.end.strftime("%Y-%m-%dT%H:%M:%SZ") + '.nc'
    print(f"downloading {filename}...")
    extPath = c3.HycomUtil.downloadToExternal(url, filename, 'hycom-tmp')
    # Move file to "local" jupyter pod filesystem
    ret1 = c3.Client.copyFilesToLocalClient(extPath, localPath)
    ret2 = c3.FileSystem.inst().deleteFiles(extPath, True)
    return localPath + '/' + filename

In [4]:
import datetime
import netCDF4 as nc
da = c3.HindcastArchive.get("GOMu0.04-expt_90.1m000-2021") # Get the 2021 Archive record
# Set options for single file download
opts = c3.HycomSubsetOptions(
    **{
        "timeRange": {
            "start": datetime.datetime(2021,5,1,12),
            "end": datetime.datetime(2021,1,5,12)
        }
    }
)

hindcast_file = downloadHindcast(da, opts)


downloading GOMu0.04-expt_90.1m000-2021-2021-05-01T12:00:00Z-2021-01-05T12:00:00Z.nc...


In [5]:
ds =nc.Dataset(hindcast_file)
print(ds)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    classification_level: UNCLASSIFIED
    distribution_statement: Approved for public release. Distribution unlimited.
    downgrade_date: not applicable
    classification_authority: not applicable
    institution: Naval Oceanographic Office
    source: HYCOM archive file
    history: archv2ncdf3z
    field_type: instantaneous
    Conventions: CF-1.6 NAVO_netcdf_v1.1
    History: Translated to CF-1.0 Conventions by Netcdf-Java CDM (CFGridWriter2)
Original Dataset = GOMu0.04/expt_90.1m000/data/hindcasts/2021; Translation Date = 2021-10-07T13:34:52.402Z
    geospatial_lat_min: 18.1200008392334
    geospatial_lat_max: 31.920000076293945
    geospatial_lon_min: -98.0
    geospatial_lon_max: -76.4000244140625
    dimensions(sizes): time(1), lat(346), lon(541), depth(40)
    variables(dimensions): int16 surf_el(time,lat,lon), float64 time(time), float64 lat(lat), float64 lon(lon), int16 salinity(time,dept